## Dependancies

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pyreadr as pyr
import pandas as pd
import tqdm
import json
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import scipy
import statsmodels
import gseapy as gp
import pyensembl as pe

print(matplotlib.__version__)
print(np.__version__)
print(pd.__version__)
print(pyr.__version__)
print(tqdm.__version__)
print(json.__version__)
print(sklearn.__version__)
print(sns.__version__)
print(scipy.__version__)
print(statsmodels.__version__)
print(gp.__version__)
print(pe.__version__)